## A simple Tephra2 workflow
Tephra2 is used to simulate tephra fallout deposits associated with explosive volcanic eruptions. In the forward solution, eruptions source parameters (ESPs) are specified, a wind field is specified, and the map locations where tephra accumulation is calculated are specified. Tephra2 produces estimates of mass loading or grainsize distribution at these map locations based on the ESPs and wind data sets. 

Explore this workflow if you are interested in:
1. Simulating an explosive eruption and generating an isomass map with Tephra2
2. Learning about ESPs and their impact on tephra accumulation
3. Creating input files for Tephra2
4. Executing compiled codes from the notebook
5. Manipulating model output with pandas and making simple contour plots.


### The workflow
The steps in this workflow are:
1. Build a configuration file for Tephra2 consisting of eruption source parameters and save this file.
2. Build a grid file for Tephra2 consisting of the points where tephra accumulation will be modeled, and save this file.
3. Build a wind file for Tephra2 consisting of wind velocity as a function of height, and save this file
4. Run Tephra2
5. Parse the Tephra2 output to obtain isomass information
6. Plot an isomass map from the parsed Tephra2 output.

### A few notes for simplicity
Unless explicitly written at the top of the cell, assume each cell needs to simply be run without alteration.


### Some Tephra2 references and examples

Bonadonna, C., Connor, C.B., Houghton, B.F., Connor, L., Byrne, M., Laing, A. and Hincks, T.K., 2005. Probabilistic modeling of tephra dispersal: Hazard assessment of a multiphase rhyolitic eruption at Tarawera, New Zealand. Journal of Geophysical Research: Solid Earth, 110(B3). <a href="https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2003JB002896">Link</a>


run <a href="https://gscommunitycodes.usf.edu/geoscicommunitycodes/public/tephra2/tephra2.php">Tephra2</a> interactively on the web

### Required installed libraries for this workflow

In [ ]:
import os  # needed to run tephra2 externally
import pandas as pd #used to manipulate tephra2 output in this notebook
import matplotlib.pyplot as plt # used to plot the isopach map
import matplotlib.tri as tri # used to plot the isopach map
import numpy as np
import cdsapi #used to pull reanalysis data
import netCDF4 #used to process downloaded netCDF format data
import subprocess #for proper program running
import utm
#Optional flag for checking Github repo for newer version
check_for_update = False

### The wind file
Tephra2 requires that the wind velcocity be specifed as a function of height above the ground surface. The code in the following cells pulls wind reanalysis data and manipulates it to give Tephra2 compatible data. Change the values to change the wind velocity as a function of height in your simulation. Any number of wind heights may be specified by adding another pressure value. 

In [ ]:
#INPUT DATE AND COORDINATE VALUES HERE

#Coordinate location
vent_latitude, vent_longitude = 22.3,33.4

#Specify month(s) (in numerical string format, e.g. "01", "11")
months = ["01"]

#Specify year(s) (in numerical string format, e.g. "1999")
years = ["2005"]

#Specify day(s) (in numerical string format, e.g. "01", "11")
days = ["04"]

#Specify hour(s) in 24 hour string format (00:00 - 23:00)
hours = ['06:00']

#Specify pressure values to generate wind field around volcano

#valid values are 
# ['1', '2', '3', '5', '7', '10', '20', '30', '50','70', '100', '125','150', '175', '200','225', '250', '300', '350', '400'
# '450','500', '550', '600','650', '700', '750','775', '800', '825','850', '875', '900','925', '950', '975','1000']
pressures = ['1','10','100','1000']

#Specify name of output file, in netCDF format (optional, default is download.nc)
file_name = []

In [ ]:
north = round(vent_latitude*4)/4
south = north +.1
east = round(vent_longitude*4)/4
west = east - .1

In [ ]:
wind_data = cdsapi.Client()
dataset = "reanalysis-era5-pressure-levels"
request = {
        "product_type": ["reanalysis"],
        "data_format": "netcdf",
        "variable": [
            "geopotential", "u_component_of_wind", "v_component_of_wind",
        ],
        "pressure_level": pressures,
        "year": years,
        "month": months,
        "day": days,
        "time": hours,
        "download_format": "unarchived",
        "area": [
            north, west, south, east
        ],
    }
wind_data.retrieve(dataset, request, "download.nc")

In [ ]:
wind = netCDF4.Dataset("download.nc")

In [ ]:
uwnd = wind["u"][0,:,0,0]
vwnd = wind["v"][0,:,0,0]

speed = np.sqrt(vwnd**2 + uwnd**2)
direction = -180/np.pi *np.arctan(vwnd/uwnd)
for d in range(len(direction)):
    if uwnd[d] > 0:
        direction[d] += 90
    else:
        direction[d] += 270

hgt = (wind["z"][0,:,0,0])/9.80665
speed = speed[::-1]
direction = direction[::-1]
hgt = hgt[::-1]

In [ ]:
row2 = []
for l in range(wind["pressure_level"].shape[0]):
    row2.append(str(hgt[l]) + ' ' + str(speed[l]) + ' ' + str(direction[l]))

# Format of data islevel (masl) speed (m/s) direction that wind is blowing toward (degrees)
wind_file = open("my_wind.dat", "w")
for element in row2:
    wind_file.write("%s\n" % element)
wind_file.close()

### The configuration file
The Tephra2 configuration file contains all the ESPs information required to run the model.

The Tephra2 code reads the configuration file and searchs for values associated with keywords. Change the values to change that eruption source parameter. Do not change the keyword.

The keywords, definitions, units, and example values are as follows:
1. VENT_EASTING: this is the easting (x) coordinate of the volcanic vent that produces the tephra. The value can be a local grid or an actual UTM grid value. Units are meters. Example: 450000 or 0
2. VENT_NORTHING: this is the northing (y) coordinate of the volcanic vent that produces the tephra. The value can be a local grid or an actual UTM grid value. Units are meters. Example: 4500000 or 0
3. VENT_ELEVATION: the elevation of the volcanic vent. Units are meters above sealevel. Example 1500
4. PLUME_HEIGHT: the height of the top of the volcanic plume. Units are meters above sealevel. Example: 12000
5. ERUPTION_MASS: the total mass of tephra produced by the eruption. Units are kg. Examples: 1e12, 10000000.
6. MEDIAN_GRAINSIZE: the median of the total grainsize distribution of the eruption. Tephra2 assumes the total grainsize distribution is normally distributed. Units are $\phi$. Examples: 0, -1, 1
7. STD_GRAINSIZE: the standard deviation of the total grainsize distribution. Units are $\phi$. Examples: 1, 2

These seven values are required for accurate modeling of tephra. The following eleven parameters (8-19) may be left empty in place of default trusted values if data is unavailable.

In [ ]:
converted = utm.from_latlon(vent_latitude,vent_longitude)

vent_easting = converted[0]

vent_northing = converted[1]

vent_elevation = 1500

plume_height = 12000

eruption_mass = 1e10

median_grain = 1

std_grain = 1

### These parameters allow for additional specificity, but can be left blank to use defaults.

8. FALL_TIME_THRESHOLD: the fall time threshold is the time at which diffusion laws switch in Tephra2, as different particle sizes have differnt diffusion in the atmosphere. Units are seconds. Examples: 1000, 1e6
9. Plume model: The plume model descibed how tephra mass is distributed in the eruption column from the vent to the total height of the volcanic plume. Options are (1) using a uniform distribution, or (2) using a beta distribution. Dimensionless.
10. PART_STEPS: Settling velocity is a function of grainsize. Tephra2 divides the total grainsize distribution into particles steps and calculates the settling velocity for each particle step. Dimensionless. Examples: 100, 200
11. DIFFUSION_COEFFICIENT: The spreading of the plume perpendicular to the major axis of dispersion is controled by the diffusion coefficient. Units are meters squared per second. Examples: 300, 3000, 300000
12. LITHIC_DENSITY: The density of small particles in the tephra. Units are kg/m^3. Examples are: 2600, 2800
13. COL_STEPS: Tephra2 discretizes the eruption column. The number of column steps is how many layers occur in the plume from the vent to the total hieght of the plume. Units are meters. Examples: 100, 200
14. MAX_GRAINSIZE: the maximum size of particles in the modeled plume. These particles must be small enough to be convected by the plume. Units are $\phi$. Examples: -4, -3.5
15. ALPHA: the alpha ($\alpha$) parameter in the Beta distribution to control the mass of tephra released as a function of height in the erupiton column (PLUME_MODEL 2). Dimensionless. Example 1, 2
16. BETA: the beta ($\beta$) parameter in the Beta distribution to control the mass of tephra released as a function of height in the erupiton column (PLUME_MODEL 2). Dimensionless. Example 1, 2
17. MIN_GRAINSIZE: the minimum size of particles in the modeled plume.  Units are $\phi$. Examples: 4, 3.5 (note: aggregation is not considered in this model)
18. PUMICE_DENSITY: The The density of large particles in the tephra. Units are kg/m^3. Examples are: 1000, 800
19. EDDY_CONST: The turbulent eddy constant for the atmosphere. Units are meters sqaured per second. Example: 0.04

In [ ]:
falltime_thresh = 0

plume_model = 0

part_steps = 0

diffusion_coef = 100000

lithic_density = 0

col_steps = 0

max_grain = 0

alpha = 0

beta = 0

min_grain = 0

pumice_density = 0

eddy_const = 0

In [ ]:
# make a simple tephra2 config file
row = []
# Note: UTM coordinates are used 
# in this example we are simply
# putting the UTM coordinates at 0,0
# Units are meters
row.append("# This is a tephra2 config file")
row.append('VENT_EASTING ' + str(vent_easting))
row.append('VENT_NORTHING ' + str(vent_northing))

#Units are meters above sea level
row.append('VENT_ELEVATION ' + str(vent_elevation))
row.append('PLUME_HEIGHT ' + str(plume_height))

# Units are kg
row.append('ERUPTION_MASS '+ str(eruption_mass))

# Units are phi
row.append('MEDIAN_GRAINSIZE ' + str(median_grain))
row.append('STD_GRAINSIZE ' + str(std_grain))

row.append("".join(('FALL_TIME_THRESHOLD ', str(1000) if not falltime_thresh else str(falltime_thresh))))
row.append("".join(('PLUME_MODEL ', str(2) if not plume_model else str(plume_model))))
row.append("".join(('PART_STEPS ', str(100) if not part_steps else str(part_steps))))
row.append("".join(('DIFFUSION_COEFFICIENT ', str(3000.0) if not diffusion_coef else str(diffusion_coef))))
row.append("".join(('LITHIC_DENSITY ', str(2600.0) if not lithic_density else str(lithic_density))))
row.append("".join(('COL_STEPS ', str(200) if not col_steps else str(col_steps))))
row.append("".join(('MAX_GRAINSIZE ', str(-4) if not max_grain else str(max_grain))))
row.append("".join(('ALPHA ', str(1) if not alpha else str(alpha))))
row.append("".join(('BETA ', str(1) if not beta else str(beta))))
row.append("".join(('MIN_GRAINSIZE ', str(4) if not min_grain else str(min_grain))))
row.append("".join(('PUMICE_DENSITY ', str(1000.0) if not pumice_density else str(pumice_density))))
row.append("".join(('EDDY_CONST ', str(0.04) if not eddy_const else str(eddy_const))))


config_file = open("my_esps.conf", "w")
for element in row:
    config_file.write(element + "\n")
config_file.close()

### The grid file
The grid file is a simple list of locations and elevations where tephra accumulation will be calculated. Note that the elevations of all points in the grid file must be the same. This is because Tephra2 using an integral solution to the advection-diffusion equation to simplify the modeling. That is, the tephra accumulates on a flat topography.

Alter the code in the following cell to create your own grid file. For now, the grid will center on the volcano/eruption point in a square.

In [ ]:
#INPUT GRID PARAMETERS HERE

#Specify easting of volcano
vol_easting = round(converted[0])
#Specify northing of volcano
vol_northing = round(converted[1])

#Specify distance around volcano (in meters, default 50km or 50,000 m)
grid_radius = []

#Specify spacing in grid (in meters, default 1km or 1,000 m)
grid_spacing = []

#Specify elevation of points, constant due to integral solution used (in meters, default 1,500 m)
elevation = []

In [ ]:
# create the tephra2 grid
#in this example, the grid is 1 km x 1km, the volcano is located
# at 0,0 (km) and the grid extends 50 km from the volcano (rectangle)

# the data in the grid file always have the form
# easting northing elevation

min_easting = vol_easting-50000 if not grid_radius else vol_easting-grid_radius
max_easting = vol_easting+50000 if not grid_radius else vol_easting+grid_radius
min_northing = vol_northing-50000 if not grid_radius else vol_northing-grid_radius
max_northing = vol_northing+50000 if not grid_radius else vol_northing+grid_radius
grid_size = 1000 if not grid_spacing else grid_spacing
elevation = 1500 if not elevation else elevation

with open("volcano_cone.grid", "w") as output_file:
    for i in range (min_easting, max_easting, grid_size):
        for j in range (min_northing, max_northing, grid_size):
            if (i != vol_easting and j != vol_northing):
                print(i, j, elevation, file = output_file)
            


### Running Tephra2
The following script runs Tephra2. The files:

1. configuration file
2. grid file
3. wind file

must be created first. See examples in previous cells in this notebook.
The command creates the tephra2.out file, in which output is stored.



In [ ]:
# this command runs tephra2
# tephra2 must be installed and accessible from the local directory
if check_for_update:
    os.chdir('../tephra2')
    subprocess.run('git pull > out.txt')
    f = open("out.txt",'r')
    line = f.readline()
    if line == "Already up to date.\n":
        os.remove('out.txt')
    else:
        subprocess.run("make")
        os.chdir("../Workflows")
subprocess.run('./../tephra2/tephra2_2020 my_esps.conf volcano_cone.grid my_wind.dat > tephra2.csv',shell=True)


### Manipulating Tephra2 output
The script in the following cell manipulates the tephra2 output. First, it reads the output file into a pandas dataframe. There is a lot of information in this dataframe. In this example, the easting, northing, and mass loading are extracted from the dataframe for each location in the grid file (where tephra accumulation was calculated). 

These extracted data are stored as lists, for ease in plotting.

In [ ]:
#use pandas to parse the output file
# in this case, ignore the granulometry and capture the tephra thickness data
# since the goal is to make an isomass map
tephra_out = pd.read_csv('tephra2.csv', delim_whitespace=True)

#extract rows from the pandas dataframe and convert to lists for easy plotting
easting = tephra_out['#EAST'].tolist()
northing = tephra_out['NORTH'].tolist()
mass = tephra_out['Kg/m^2'].tolist()


### Plotting the isomass map
The following script is an example of one way to plot a Tephra2 isomass data.

In [ ]:
#SET DISPLAY INPUTS HERE

#Specify levels in which solid countour lines should be generated
levels_solid = []

#Specify intermediate dashed lines for contour
levels_dashed = []

#specify filetype (jpg, png, pdf, svg)
fileext = "svg"

In [ ]:
# plot a simple isomass map
import cartopy.crs as ccrs
from cartopy.io.img_tiles import Stamen
import matplotlib.pyplot as plt

# plot contours in kg/m^2 with solid lines
cntr2 = plt.tricontour(easting, northing, mass, levels=[1,10,100,1000,10000] if not levels_solid else levels_solid,
        colors=('k',),linestyles=('-',),linewidths=(1,))

#plot contours in kg/m^2 with dashed lines
cntr3 = plt.tricontour(easting, northing, mass, levels=[5,50,500] if not levels_dashed else levels_dashed,
        colors=('k',),linestyles=('dashed',),linewidths=(0.5,))

#label contours of solid lines
plt.clabel(cntr2, fmt = '%2.1d', colors = 'k', fontsize=6) #contour line labels

#plot the volcano location
plt.plot(vol_easting,vol_northing, "k^")

#map the map square
plt.axis("square")
plt.rcParams["figure.figsize"]=(20, 20)
plt.xlabel("Easting (m)")
plt.ylabel("Northing (m)")
plt.savefig("tephra_contour."+fileext)
plt.grid(True)
plt.show()